<a href="https://colab.research.google.com/github/Muppasaisrija/Muppa_NFO5731_-SPRING2022/blob/main/INFO5731_Assignment_Three_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [ ]:
# Write your code here

import os
import tweepy as tw
import pandas as pd
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('words')
from nltk.corpus import stopwords
import requests
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer

import nltk
nltk.download('omw-1.4')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
words = set(nltk.corpus.words.words())

def get_lemmatized_docs(X):
    stemmer = WordNetLemmatizer()
    eng_stop_word_list = stopwords.words("english")
    bag_of_words = []
    for sen in range(0, len(X)):
    # removing all the special characters
        document = re.sub(r'\W', ' ', str(X[sen]))

      # removing all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

      # removing multiple spaces and adding single space in place
        document = re.sub(r'\s+', ' ', document, flags=re.I)

      # removing the 'b' prefix added during byte conversion
        document = re.sub(r'^b\s+', '', document)

      # removing numbers
        document = re.sub(r'\d+','',document)

      # converting all words to lowercase
        document = document.lower()
      
      # Splitting the cleaned document into a list of words, which will be subsequently added to a bag after more processing
        document = document.split()
        document = [stemmer.lemmatize(word) for word in document]
        document = [word for word in document if word not in eng_stop_word_list]
        document = ' '.join(document)

        document = " ".join(word for word in nltk.wordpunct_tokenize(document) if word.lower() in words or not word.isalpha())

      #Bag of words
        bag_of_words.append(document)

    return bag_of_words

df = pd.read_csv('/content/drive/MyDrive/Nokia Reviews.csv')
#df.dropna(how='any', inplace=True)
clean_docs  = get_lemmatized_docs(df["Review"])
print(len(clean_docs))
print(clean_docs)

In [ ]:
words = [word for word in clean_docs if str(word) != 'nan' and str(word) != '']
print(words)
word_list = []
for word in words:
  [word_list.append(val) for val in word.split(' ')]

word_str = ''
for word in word_list:
  word_str += word + ' '
print(word_str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """


[]

In [ ]:
import nltk
nltk.download('punkt')
token_list = nltk.word_tokenize(word_str)
res_trigrams = nltk.trigrams(token_list)
freq_trigrams = nltk.FreqDist(res_trigrams)
freq_trigrams

In [ ]:
from nltk.util import ngrams
import collections
token_bigram = nltk.word_tokenize(word_str)
bigrams = ngrams(token_bigram,2)
bigram_dist = nltk.FreqDist(bigrams)
single_word_dist = nltk.FreqDist([word for word in word_str.split(' ')])

for w1,w2 in bigram_dist:
    count_w1_w2 = bigram_dist[(w1,w2)]
    count_w1 = single_word_dist[w1]
    if(count_w1 != 0):
        print(w1,w2,(count_w1_w2/count_w1))

In [ ]:
from typing import AnyStr

from collections import Counter      
import nltk
from nltk import word_tokenize, pos_tag, pos_tag_sents

import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Nokia Reviews.csv')

nltk.download('averaged_perceptron_tagger')
lower_case = word_str.lower()
word_tokens = nltk.word_tokenize(lower_case)
parts_of_speech_tags = nltk.pos_tag(word_tokens)

df.dropna(how='any', inplace=True)
cnts = Counter( tag for word,  tag in parts_of_speech_tags)
max_cnt = cnts['NN']
df['word groups'] = pos_tag_sents(df['Review'].apply(word_tokenize).tolist())


word_grps = df['word groups']
first = []
scnd = []
for grp in word_grps:
    cnt = Counter( tag for word,  tag in grp)
    first.append(cnt['NN'])
for val in first:
    rel_prob = val/max_cnt
    scnd.append(rel_prob)
print(scnd)

df1 = pd.DataFrame(first,columns = ['noun_count'])
df2 = pd. DataFrame(scnd,columns = ['prob_noun'])
res = pd.concat([df1,df2], axis=1, sort=False)
res_frame = pd.concat([df,res], axis = 1, sort = False)
res_frame

# **Question 2: Undersand TF-IDF and Document representation**

(20 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [ ]:
# Write your code here\
from sklearn.feature_extraction.text import TfidfVectorizer
text = []
for x in df['Review']:
    text.append(x)
vectorizer = TfidfVectorizer()
tf_idf_matrix = vectorizer.fit_transform(text)
attributes = vectorizer.get_feature_names()
index=[n for n in range(1, len(text)+1)]
vectorizer = TfidfVectorizer(input=text, min_df=0.1, stop_words='english')
tf_idf=pd.DataFrame(tf_idf_matrix.T.todense(), index=attributes, columns=index)
tf_idf

In [ ]:
query = "Take these reviews with a pinch of salt."
cleaned_query = get_lemmatized_docs([query])
print(cleaned_query[0])

In [ ]:
#cosine similarity
from nltk import corpus
import pandas
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
text = []
for x in df['Review']:
    text.append(x)

text.append(cleaned_query[0])
vectorizer = TfidfVectorizer(input=text, min_df=0.1, stop_words='english')
matrix = vectorizer.fit_transform(text)

cs_dict = {}
for i in range(0, matrix.shape[0]):
    sim=cosine_similarity(matrix[80], matrix[i])
    cs_dict[str(i)] = sim[0][0]
    i+=1

sorted_keys = (sorted(cs_dict, key=cs_dict.get,reverse=True))
sorted_cosine_dict = {}
for key in sorted_keys:
    sorted_cosine_dict[key] = cs_dict[key];

Review = []
cos = []
for y in sorted_cosine_dict:
    text.append([int(y)])
    cos.append(sorted_cosine_dict[y])
dafrme1 = pd.DataFrame(Review)

dafrme1['Rank'] = [i for i in range(1, len(sorted_keys)+1)]
dafrme1['Cossim'] = cos
dafrme1.head(500)

# **Question 3: Create your own word embedding model**

(20 points). Use the data you collected for assignment two to build a word embedding model: 

(1) Train a 300-dimension word embedding (it can be word2vec, glove, ulmfit, bert, or others).

(2) Visualize the word embedding model you created.

Reference: https://machinelearningmastery.com/develop-word-embeddings-python-gensim/

In [ ]:
# Write your code here

!pip install --upgrade gensim

import gensim
from gensim import models

from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot

In [ ]:
docs = [word.split() for word in clean_docs if str(word) != 'nan' and str(word) != '']
model = models.Word2Vec(docs,vector_size=300)
words = list(model.wv.index_to_key)
print(words)

In [ ]:
model.wv.save_word2vec_format('model.bin')
#model.wv.save_word2vec_format('model.txt', binary=False)
model.save('model.bin')
print(model)

In [ ]:
X = model.wv[model.wv.index_to_key]
pca = PCA(n_components=2)
result = pca.fit_transform(X)
# create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1])
words = list(model.wv.index_to_key)
for i, word in enumerate(words):
	pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()

# **Question 4: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [ ]:
# The GitHub link of your final csv file

https://github.com/smmohua/INFO-5731/blob/main/tweet_sentiment_analysis.csv

# Link: 




